# Домашнее задание к вебинару "Связь бизнес-показателей и DS-метрик"

### Импорт библиотек

In [9]:
import pandas as pd
import numpy as np
from gensim.corpora.dictionary import Dictionary
import re
import numpy as np
from gensim.models import LdaModel
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.linear_model import LogisticRegression
from loaders import ColumnSelector
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
# from catboost import CatBoostClassifier
from xgboost import XGBClassifier

from my_utils import lemmatization, clean_text

RANDOM_STATE = 42

## Задание 1

обучить несколько разных моделей на наборе данных ССЗ (train_case2.csv): логрег, бустинг, лес и т.д - на ваш выбор 2-3 варианта

In [10]:
comments = pd.read_csv("../lection/train.csv")
print(comments.shape)
comments.head(3)

(159571, 8)


,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0


Сначала нужно получить векторные представления комментариев

In [11]:
%%time
# Запускаем очистку текста. Будет долго...
comments['comment_text'] = comments['comment_text'].apply(lambda x: clean_text(x), 1)

Wall time: 22.5 s


In [12]:
comments.head(5)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,explanation why the edits made under my userna...,0,0,0,0,0,0
1,000103f0d9cfb60f,d'aww he matches this background colour i'm se...,0,0,0,0,0,0
2,000113f07ec002fd,hey man i'm really not trying to edit war it's...,0,0,0,0,0,0
3,0001b41b1c6bb37e,""" more i can't make any real suggestions on im...",0,0,0,0,0,0
4,0001d958c54c6e35,you sir are my hero any chance you remember wh...,0,0,0,0,0,0


In [13]:
%%time
#Запускаем лемматизацию текста. Будет очень долго...
comments['comment_text'] = comments['comment_text'].apply(lambda x: ' '.join(lemmatization(x)), 1)

Wall time: 2min 46s


In [14]:
comments.head(5)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,explanation edits username hardcore metallica ...,0,0,0,0,0,0
1,000103f0d9cfb60f,aww matches background colour seemingly stuck ...,0,0,0,0,0,0
2,000113f07ec002fd,hey man trying edit war guy constantly removin...,0,0,0,0,0,0
3,0001b41b1c6bb37e,real suggestions improvement wondered section ...,0,0,0,0,0,0
4,0001d958c54c6e35,sir hero chance remember page,0,0,0,0,0,0


In [16]:
common_processing_pipes = [('title_selector', ColumnSelector(key='comment_text')), 
                           ('title_tfidf', TfidfVectorizer())]

log_reg_pipeline = Pipeline(common_processing_pipes + [('clf', LogisticRegression(C=0.1, solver='sag'))])
# catboost_pipeline = Pipeline(common_processing_pipes + [('clf', CatBoostClassifier(
#     class_weights=[1, 3.5], 
#     silent=True, 
#     random_state=RANDOM_STATE
# ))])
xgboost_pipeline = Pipeline(common_processing_pipes + [('clf', XGBClassifier(random_state=RANDOM_STATE))])

2. при обучении моделей обязательно использовать кроссвалидацию

In [17]:
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score

y = comments['toxic']
X = comments.drop(columns=['toxic'])
n_samples = X.shape[0]
cv = ShuffleSplit(n_splits=5, test_size=0.3, random_state=0)

In [18]:
log_reg_precision_scores = cross_val_score(log_reg_pipeline, X, y, cv=cv, scoring="precision_macro")
log_reg_recall_scores = cross_val_score(log_reg_pipeline, X, y, cv=cv, scoring="recall_macro")
log_reg_roc_auc_scores = cross_val_score(log_reg_pipeline, X, y, cv=cv, scoring="roc_auc")
log_reg_f1_scores = cross_val_score(log_reg_pipeline, X, y, cv=cv, scoring="f1_macro")

In [22]:
xgboost_precision_scores = cross_val_score(xgboost_pipeline, X, y, cv=cv, scoring="precision_macro")
xgboost_recall_scores = cross_val_score(xgboost_pipeline, X, y, cv=cv, scoring="recall_macro")
xgboost_roc_auc_scores = cross_val_score(xgboost_pipeline, X, y, cv=cv, scoring="roc_auc")
xgboost_f1_score_scores = cross_val_score(xgboost_pipeline, X, y, cv=cv, scoring="f1_macro")

c:\python39\lib\site-packages\xgboost\sklearn.py:892: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[11:30:30] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[11:30:31] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[11:31:22] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip

c:\python39\lib\site-packages\xgboost\sklearn.py:892: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[11:40:08] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[11:40:09] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


c:\python39\lib\site-packages\xgboost\sklearn.py:892: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


...

[11:41:41] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[11:41:42] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


c:\python39\lib\site-packages\xgboost\sklearn.py:892: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[11:42:27] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[11:42:28] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


c:\python39\lib\site-packages\xgboost\sklearn.py:892: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[11:43:18] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[11:43:19] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[11:44:09] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip

3. вывести сравнение полученных моделей по основным метрикам классификации: pr/rec/auc/f_score (можно в виде таблицы, где строки - модели, а столбцы - метрики)

In [25]:
data = {
    'logistic_regression': [
        np.mean(log_reg_precision_scores),
        np.mean(log_reg_recall_scores),
        np.mean(log_reg_roc_auc_scores),
        np.mean(log_reg_f1_scores)
    ],
    'xgboost': [
        np.mean(xgboost_precision_scores),
        np.mean(xgboost_recall_scores),
        np.mean(xgboost_roc_auc_scores),
        np.mean(xgboost_f1_score_scores)
    ]
}
pd.DataFrame.from_dict(data, orient='index', columns=['precision', 'recall', 'roc_auc', 'f1_score'])

,precision,recall,roc_auc,f1_score
logistic_regression,0.958986,0.653344,0.955585,0.716288
xgboost,0.927360,0.781352,0.944336,0.835696


4. сделать выводы о том, какая модель справилась с задачей лучше других

Модели примерно одинаково хорошо справляются с задачей

5. (опциональный вопрос) какая метрика (precision_recall_curve или roc_auc_curve) больше подходит в случае сильного дисбаланса классов? (когда объектов одного из классов намного больше чем другого).

precision_recall_curve больше подходит